In [1]:
import os
os.chdir("../")
%pwd

'd:\\Machine_Learning\\Consignment-Pricing-Prediction'

In [14]:
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    scaler_path: Path
    metrics_file_path: Path
    target_column: str

In [15]:
from ConsignmentPricingPrediction.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from ConsignmentPricingPrediction.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH,
            ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)->ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            scaler_path = config.scaler_path,
            metrics_file_path = config.metrics_file_path,
            target_column = params.TARGET_COLUMN,
        )
        return model_evaluation_config

In [43]:
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from ConsignmentPricingPrediction.utils.common import load_object
from ConsignmentPricingPrediction.logging import Logger
import yaml

class ModelEvaluation:
    def __init__(self, model_evaluation_config: ModelEvaluationConfig):
        self.config = model_evaluation_config

    def fetch_dataset(self):
        data = pd.read_csv(self.config.test_data_path)
        Logger.info("Test Dataset has been fetched successfully")
        return data
    
    def seperating_dataset(self, testDF: pd.DataFrame):
        x = testDF.drop(columns=[self.config.target_column], axis=1)
        y = testDF[self.config.target_column]

        Logger.info('Test Dataset has been separated successfully')
        return x,y
    
    def scal_dataset(self, testDF: pd.DataFrame):
        scaler = load_object(path= Path(self.config.scaler_path))
        scaled_testDF = scaler.transform(testDF)

        Logger.info('Test Dataset has been scaled successfully')
        return scaled_testDF
    
    def fetch_model(self):
        model = load_object(path= Path(self.config.model_path))

        Logger.info('Trained Model has been fetched successfully')
        return model
    
    
    def save_metrics(self, r2, mae, mse, mape):
        metrics = {
            "R2_Score": float(r2),
            "Mean_Absolute_Error": float(mae),
            "Mean_Squared_Error": float(mse),
            "Mean_Absolute_Percentage_Error": float(mape),
        }

        with open(self.config.metrics_file_path, 'w') as f:
            yaml.dump(metrics, f)
        Logger.info('Trained Metrics has been saved successfully at {}'.format(self.config.metrics_file_path))

    def evaluate_model(self):
        data = self.fetch_dataset()
        x,y =  self.seperating_dataset(data)
        scal_x = self.scal_dataset(x)

        model = self.fetch_model()

        y_pred = model.predict(scal_x)
        Logger.info("Predictions on the testing dataset has been computed")

        r2 = r2_score(y, y_pred)
        mae = mean_absolute_error(y, y_pred)
        mse = mean_squared_error(y,y_pred)
        mape = mean_absolute_percentage_error(y,y_pred)
        Logger.info("Model has been evaluated")
        
        self.save_metrics(r2, mae, mse, mape)
        

In [44]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.evaluate_model()
except Exception as e:
    raise e

[2024-03-22 13:52:52,679: INFO: common: yaml file config\config.yaml loaded successfully]
[2024-03-22 13:52:52,684: INFO: common: yaml file params.yaml loaded successfully]
[2024-03-22 13:52:52,684: INFO: common: created directory at: artifacts]
[2024-03-22 13:52:52,684: INFO: common: created directory at: artifacts/model_evaluation]
[2024-03-22 13:52:52,697: INFO: 1731941409: Test Dataset has been fetched successfully]
[2024-03-22 13:52:52,699: INFO: 1731941409: Test Dataset has been separated successfully]
[2024-03-22 13:52:52,703: INFO: 1731941409: Test Dataset has been scaled successfully]
[2024-03-22 13:52:52,707: INFO: 1731941409: Trained Model has been fetched successfully]
[2024-03-22 13:52:52,714: INFO: 1731941409: Predictions on the testing dataset has been computed]
[2024-03-22 13:52:52,714: INFO: 1731941409: Model has been evaluated]
[2024-03-22 13:52:52,724: INFO: 1731941409: Trained Metrics has been saved successfully at artifacts/model_evaluation/metrics.yaml]
